# Basic text search app with simplified API

> The simplified API minimizes the work required to create simple apps involving only one document type.

The first step is to create a Vespa [ApplicationPackage](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.package.ApplicationPackage):

In [1]:
from vespa.package import ApplicationPackage

app_package = ApplicationPackage(name="cord19")

### Add fields to the Schema

We can then add [fields](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.package.Field) to the application's [Schema](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.package.Schema) created by default in `app_package`.

In [2]:
from vespa.package import Field

app_package.schema.add_fields(
    Field(name = "cord_uid", type = "string", indexing = ["attribute", "summary"]),
    Field(name = "title", type = "string", indexing = ["index", "summary"], index = "enable-bm25"),
    Field(name = "abstract", type = "string", indexing = ["index", "summary"], index = "enable-bm25")
)

* `cord_uid` will store the cord19 document ids, while `title` and `abstract` are self explanatory. 

* All the fields in this case are of type `string`. 

* Including `"index"` in the `indexing` list means that Vespa will create a searchable index for `title` and `abstract`. You can read more about which options is available for `indexing` in the [Vespa documentation](https://docs.vespa.ai/documentation/reference/schema-reference.html#indexing). 

* Setting `index = "enable-bm25"` makes Vespa pre-compute quantities to make it fast to compute the bm25 score. We will use BM25 to rank the documents retrieved.

### Search multiple fields when querying

A [Fieldset](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.package.FieldSet) groups fields together for searching. For example, the `default` fieldset defined below groups `title` and `abstract` together.

In [3]:
from vespa.package import FieldSet

app_package.schema.add_field_set(
    FieldSet(name = "default", fields = ["title", "abstract"])
)

### Define how to rank the documents matched

We can specify how to rank the matched documents by defining a [RankProfile](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.package.RankProfile). In this case, we defined the `bm25` rank profile that combines that BM25 scores computed over the `title` and `abstract` fields. 

In [4]:
from vespa.package import RankProfile

app_package.schema.add_rank_profile(
    RankProfile(name = "bm25", first_phase = "bm25(title) + bm25(abstract)")
)